# [초급 프로젝트] 4팀_김명환

---
---

# 프로그래밍

In [1]:
!pip install -q gdown
!pip install -q albumentations
!pip install -q ultralytics
!pip install -q -U ultralytics
!pip install -q nbformat
!pip install -q roboflow
!pip install -q opencv-python
!pip install -q opencv-python-headless
print("로딩완료")

로딩완료


In [2]:
# 기본 라이브러리 (중복 제거 및 정리)

# --- Scikit-learn: 데이터 전처리, 모델, 평가 ---
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import (
    fetch_california_housing, load_iris, make_moons, make_circles,
    load_breast_cancer, load_wine
)
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, mean_squared_error, average_precision_score

# --- 이미지 처리 ---
import cv2
from PIL import Image, ImageFilter, ImageDraw
import albumentations as A

# --- PyTorch: 딥러닝 관련 ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, Subset
# 문제 있는 v2 import 제거하고 필요시에만 개별적으로 import
# from torchvision.transforms import v2, functional as TF
from torchvision.transforms import functional as TF
from torchvision.datasets import CocoDetection
from torch.nn import CrossEntropyLoss
from collections import OrderedDict

# --- COCO 데이터셋 관련 ---
from pycocotools.coco import COCO
from pycocotools import mask as coco_mask

# --- 딥러닝 모델 ---
import timm

# --- 기본 라이브러리 ---
import os
import sys
import re
import csv
import copy
import json
import math
import random
import yaml
import shutil
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

# --- 데이터 분석 및 시각화 ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- 시간 관련 ---
from datetime import datetime, timezone, timedelta
import pytz

# --- 진행률 표시 ---
import IPython.display
from tqdm.notebook import tqdm

# --- 시간대 설정 ---
__kst = pytz.timezone('Asia/Seoul')

# --- GPU 설정 ---
__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
__device_cpu = torch.device('cpu')

# --- 재현 가능한 결과를 위한 시드 설정 ---
np.random.seed(42)
torch.manual_seed(42)
if __device.type == 'cuda':
    torch.cuda.manual_seed_all(42)

print(f"라이브러리 로드 완료 사용장치: {__device}")

라이브러리 로드 완료 사용장치: cpu


In [3]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import importlib
import helper_c0z0c_dev as helper
importlib.reload(helper)

🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.1.3 (v2.x+): 호환성 모드 적용됨
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_health_eat\scripts\김명환
🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.1.3 (v2.x+): 호환성 모드 적용됨
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_health_eat\scripts\김명환


<module 'helper_c0z0c_dev' from 'd:\\GoogleDrive\\codeit_ai_health_eat\\scripts\\김명환\\helper_c0z0c_dev.py'>

In [4]:
import os, sys
from pathlib import Path

utils_dir = None
if helper.is_colab:
    utils_dir = "/content/drive/MyDrive/codeit_ai_health_eat/src/python_modules/utils"
else:
    utils_dir = os.path.join(Path.cwd().drive + '\\', 'GoogleDrive', "codeit_ai_health_eat", "src", "python_modules", "utils")

print("utils_dir:", utils_dir)

sys.path.append(str(utils_dir))
print("sys.path:", sys.path)
import importlib
import health_ea_utils as heu
importlib.reload(heu)
from health_ea_utils import *

print("helper.__file__:", helper.__file__)
print("health_ea_utils.__file__:", heu.__file__)


utils_dir: d:\GoogleDrive\codeit_ai_health_eat\src\python_modules\utils
sys.path: ['c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\python310.zip', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\DLLs', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827', '', 'C:\\Users\\sw1\\AppData\\Roaming\\Python\\Python310\\site-packages', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\win32', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\win32\\lib', 'c:\\Users\\sw1\\anaconda3\\envs\\env_colab_250827\\lib\\site-packages\\Pythonwin', 'd:\\GoogleDrive\\codeit_ai_health_eat\\src\\python_modules\\utils']
🌐 https://c0z0c.github.io/jupyter_hangul
ℹ️ NumPy 2.1.3 (v2.x+): 호환성 모드 적용됨
✅ 설정 완료: 한글 폰트, plt 전역 등록, pandas 확장, 캐시 기능
pd commit 저장 경로 = d:\GoogleDrive\codeit_ai_health_eat\scripts\김명환
🌐 https://c0z0c.github.io/ju

# 1. 학습용 데이타 다운로드 및 압축 풀기

In [5]:
def get_tqdm_kwargs():
    """Widget 오류를 방지하는 안전한 tqdm 설정"""
    return {
        'disable': False,
        'leave': True,
        'file': sys.stdout,
        'ascii': True,  # ASCII 문자만 사용
        'dynamic_ncols': False,
#        'ncols': 80  # 고정 폭
    }

def drive_root():
    root_path = os.path.join("D:\\", "GoogleDrive")
    if helper.is_colab:
        root_path = os.path.join("/content/drive/MyDrive")
    return root_path

def get_path_modeling(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def get_path_modeling_release(add_path = None):
    modeling_path = "modeling_yolo"
    path = os.path.join(drive_root(),modeling_path)
    if add_path is not None:
        path = os.path.join(path,add_path)
    return path

def print_dir_tree(root, max_depth=2, list_count=3, indent=""):
    import os
    if max_depth < 0:
        return
    try:
        items = os.listdir(root)
    except Exception as e:
        print(indent + f"[Error] {e}")
        return

    img_count = len([f for f in os.listdir(root) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
    for item in items:
        path = os.path.join(root, item)
        if os.path.isdir(path):
            print(indent + "|-- "+ item)
            # 이미지 파일 개수만 출력
            img_count = len([f for f in os.listdir(path) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt'))])
            if img_count > list_count:
                print(indent + "   "+ f"[데이터파일: {img_count}개]")
            print_dir_tree(root=path, max_depth=max_depth-1, list_count=list_count, indent=indent + "   ")
        else:
            if list_count < img_count and item.lower().endswith(('.jpg', '.jpeg', '.png', '.xml', '.inf', '.txt')):
                continue
            print(indent + "|-- "+ item)

def save_model_dict(model, path, pth_name, kwargs=None):
    """모델 state_dict와 추가 정보를 저장"""
    def safe_makedirs(path):
        """안전한 디렉토리 생성"""
        if os.path.exists(path) and not os.path.isdir(path):
            os.remove(path)  # 파일이면 삭제
        os.makedirs(path, exist_ok=True)

    # 디렉토리 생성
    safe_makedirs(path)

    # 모델 구조 정보 추출
    model_info = {
        'class_name': model.__class__.__name__,
        'init_args': {},
        'str': str(model),
        'repr': repr(model),
        'modules': [m.__class__.__name__ for m in model.modules()],
    }

    # 생성자 인자 자동 추출(가능한 경우)
    if hasattr(model, '__dict__'):
        for key in ['in_ch', 'base_ch', 'num_classes', 'out_ch']:
            if hasattr(model, key):
                model_info['init_args'][key] = getattr(model, key)

    # kwargs 처리
    extra_info = {}
    if kwargs is not None:
        if isinstance(kwargs, str):
            extra_info = json.loads(kwargs)
        elif isinstance(kwargs, dict):
            extra_info = kwargs

    model_info.update(extra_info)

    # 저장할 dict 구성
    save_dict = {
        'model_state': model.state_dict(),
        'class_name': model.__class__.__name__,
        'model_info': model_info,
    }

    save_path = os.path.join(path, f"{pth_name}.pth")
    torch.save(save_dict, save_path)
    return save_path

def load_model_dict(path, pth_name=None):
    """
    save_model_dict로 저장한 모델을 불러오는 함수
    반환값: (model_state, model_info)
    """
    import torch
    load_path = path
    if pth_name is not None:
        load_path = os.path.join(path, f"{pth_name}.pth")
    checkpoint = torch.load(load_path, map_location='cpu', weights_only=False)  # <-- 여기 추가
    model_state = checkpoint.get('model_state')
    model_info = checkpoint.get('model_info')
    model_info['file_name'] = os.path.basename(load_path)
    return model_state, model_info


def search_pth_files(base_path):
    """
    입력된 경로의 하위 폴더들에서 pth 파일들을 검색
    """
    pth_files = []

    if not os.path.exists(base_path):
        print(f"경로가 존재하지 않습니다: {base_path}")
        return pth_files

    print(f"pth 파일 검색 시작: {base_path}")

    # 하위 폴더들을 순회하며 pth 파일 검색
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.pth'):
                pth_path = os.path.join(root, file)
                pth_files.append(pth_path)

    # 결과 정리 및 출력
    if pth_files:
        print(f"\n발견된 pth 파일들 ({len(pth_files)}개):")
        for i, pth_file in enumerate(pth_files, 1):
            # 상대 경로로 표시 (base_path 기준)
            rel_path = os.path.relpath(pth_file, base_path)
            print(f" {i:2d}. {rel_path}")
    else:
        print("pth 파일을 찾을 수 없습니다.")

    return pth_files

def print_json_tree(data, indent="", max_depth=4, _depth=0, list_count=2, print_value=True):
    """
    JSON 객체를 지정한 단계(max_depth)까지 트리 형태로 출력
    - list 타입은 3개 이상일 때 개수만 출력
    - 하위 노드가 값일 경우 key(type) 형태로 출력
    - print_value=True일 때 key(type): 값 형태로 출력
    """
    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            if isinstance(value, (dict, list)):
                print(f"{indent}|-- {key}")
                print_json_tree(value, indent + "    ", max_depth, _depth + 1, list_count, print_value)
            else:
                if print_value:
                    print(f"{indent}|-- {key}({type(value).__name__}): {value if len(str(value)) < 100 else f'{str(value)[:30]}...'}")
                else:
                    print(f"{indent}|-- {key}({type(value).__name__})")
    elif isinstance(data, list):
        if len(data) > list_count:
            print(f"{indent}|-- [list] ({len(data)} items)")
        else:
            for i, item in enumerate(data):
                if isinstance(item, (dict, list)):
                    print(f"{indent}|-- [{i}]")
                    print_json_tree(item, indent + "    ", max_depth, _depth + 1, list_count, print_value)
                else:
                    if print_value:
                        print(f"{indent}|-- [{i}]({type(item).__name__}): {item if len(str(item)) < 100 else f'{str(item)[:30]}...'}")
                    else:
                        print(f"{indent}|-- [{i}]({type(item).__name__})")
    else:
        if print_value:
            print(f"{indent}{type(data).__name__}: {data if len(str(data)) < 100 else f'{str(data)[:30]}...'}")
        else:
            print(f"{indent}{type(data).__name__}")

def print_git_tree(data, indent="", max_depth=3, _depth=0):
    """
    PyTorch tensor/딕셔너리/리스트를 git tree 스타일로 출력
    """
    import torch
    import numpy as np

    if _depth > max_depth:
        return
    if isinstance(data, dict):
        for key, value in data.items():
            print(f"{indent}├─ {key} [{type(value).__name__}]")
            print_git_tree(value, indent + "│  ", max_depth, _depth + 1)
    elif isinstance(data, (list, tuple)):
        for i, item in enumerate(data):
            print(f"{indent}├─ [{i}] [{type(item).__name__}]")
            print_git_tree(item, indent + "│  ", max_depth, _depth + 1)
    elif torch.is_tensor(data):
        shape = tuple(data.shape)
        dtype = str(data.dtype)
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ Tensor shape={shape} dtype={dtype} preview={preview_str}")
    elif isinstance(data, np.ndarray):
        shape = data.shape
        dtype = data.dtype
        preview = str(data)
        preview_str = preview[:80] + ("..." if len(preview) > 80 else "")
        print(f"{indent}└─ ndarray shape={shape} dtype={dtype} preview={preview_str}")
    else:
        val_str = str(data)
        print(f"{indent}└─ {type(data).__name__}: {val_str[:80]}{'...' if len(val_str)>80 else ''}")


print("유틸리티 함수 로드 완료")

유틸리티 함수 로드 완료


## YOLO 모델링

In [6]:
from pathlib import Path
from ultralytics import YOLO

# GPU 확인
device = __device
print(f"사용 디바이스: {device}")
print(f"CUDA 버전: {torch.version.cuda}")

사용 디바이스: cpu
CUDA 버전: None


### 3.3 모델링 엔진

In [225]:
import pytz

class PillAnalysisEngine:
    def __init__(self):
        self.DEBUG_ON = True
        # --- 시간대 설정 ---
        self.__kst = pytz.timezone('Asia/Seoul')

        # --- GPU 설정 ---
        self.__device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.__device_cpu = torch.device('cpu')
        self.project_path = os.path.join(drive_root(), "codeit_ai_health_eat")
        self.modeling_path = os.path.join(self.project_path, "src", "python_modules", "modeling")
        self.data_path = os.path.join(self.project_path, "src", "python_modules", "data")
        
        if self.DEBUG_ON:
            print(self.project_path)
            print(self.modeling_path)
            print(self.data_path)
            
        self.database = self.init_database() # DB 초기화 클래스 개수 확인을 위하여 가장 먼저 로딩되어야함.
        self.model_1way = self.load_1way_model()
        self.model_2way = self.load_2way_model()
    
        self.transform = transforms.Compose([
                transforms.Resize((300, 300)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225])
            ])
    
    def analyze_image(self, image) -> dict:
        """입력 이미지를 분석하여 약물(알약) 탐지 및 분류를 수행합니다.
        Args:
            image
                문자열 파일 경로 (str): 이미지 파일의 경로
                numpy 배열 (np.ndarray): 이미지 데이터 (예: cv2.imread()로 읽은 배열)
                PIL 이미지 객체 (PIL.Image.Image)
                torch.Tensor: 이미지 텐서 (채널 순서 및 정규화 필요)            

        Returns:
            dict: 분석 결과
        """
        
        
        # # 1단계: 이미지 검증
        # validated_image = await self.validate_image(image)
        
        # # 2단계: 객체 탐지
        # detections = await self.detect_pills(validated_image)
        
        # # 3단계: 객체 분류
        # classifications = await self.classify_pills(validated_image, detections)
        
        # # 4단계: 결과 검증 및 DB 조회
        # results = await self.validate_and_enrich(classifications)

        # 1단계: 이미지 검증
        validated_image = self.validate_image(image)
        
        # 2단계: 객체 탐지
        detections = self.detect_pills(validated_image)
        
        # 3단계: 객체 분류
        classifications = self.classify_pills(validated_image, detections)
        
        # 4단계: 결과 검증 및 DB 조회
        df_results = self.validate_and_enrich(classifications)
        
        return self.format_response(image, df_results, classifications)
        
    def find_ddi(self, category_ids):
        """
        한글 성분명 리스트를 받아서 병용금기 약물 조합을 찾는 함수
        
        Args:
            df: 병용금기 데이터프레임
            korean_ingredient_list: 사진에서 인식된 한글 성분명 리스트
        
        Returns:
            병용금기 조합 정보가 담긴 리스트 (중복 제거됨)
        """
        df_drug = self.database['df_drug']
        df = self.database['td_interaction']
        
        df_drug = df_drug[df_drug['category_id'].isin(category_ids)]
        # df_drug.head_att(10)
        
        # print('-' * 80)

        df_drug['code'] = df_drug['di_edi_code'].str.split(',').str[0].astype(int)
        codes = df_drug['code'].tolist()
        code_to_category = dict(zip(df_drug['code'], df_drug['category_id']))
        
        df_ddi = pd.DataFrame()        
        df_drug_ddi = pd.DataFrame()
        for code_a in codes:
            mask_a = df['제품코드A'] == code_a
            for code_b in codes:
                mask_b = df['제품코드B'] == code_b
                if code_a == code_b: continue
                # 매칭되는 행들을 가져오기
                matched_rows = df[mask_a & mask_b].copy()
                if not matched_rows.empty:
                    # category_ids 컬럼 추가
                    matched_rows['category_id'] = code_to_category.get(code_a, None)
                    df_ddi = pd.concat([df_ddi, matched_rows], axis=0)
                
                # ddi_drug에도 추가
                matched_drug_rows_a = df[mask_a].copy()
                if not matched_drug_rows_a.empty:
                    matched_drug_rows_a['category_id'] = code_to_category.get(code_a, None)
                    df_drug_ddi = pd.concat([df_drug_ddi, matched_drug_rows_a], axis=0)

                matched_drug_rows_b = df[mask_b].copy()
                if not matched_drug_rows_b.empty:
                    matched_drug_rows_b['category_id'] = code_to_category.get(code_b, None)
                    df_drug_ddi = pd.concat([df_drug_ddi, matched_drug_rows_b], axis=0)

        result = {}
        result['ddi'] = df_ddi.drop_duplicates().copy()
        result['ddi_drug'] = df_drug_ddi.drop_duplicates().copy()
        
        # print('ddi=',len(result['ddi']))
        # result['ddi'].head_att(20)
        # print('-' * 80)
        # print('ddi_drug=',len(result['ddi_drug']))
        # result['ddi_drug'].head_att(20)
        # print('=' * 80)
        
        return result

    def validate_image(self, image) -> bool:
        """입력 이미지를 검증합니다.
        Args:
            image: 검증할 이미지

        Returns:
            bool: 이미지 검증 결과
        """
        # image가 경로 일수 있음.
        # PIL 이미지로 변환
        if isinstance(image, str):
            image = Image.open(image).convert("RGB")
        elif isinstance(image, np.ndarray):
            image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        elif isinstance(image, torch.Tensor):
            image = TF.to_pil_image(image)
        elif not isinstance(image, Image.Image):
            raise ValueError("지원되지 않는 이미지 형식입니다. (str, np.ndarray, PIL.Image.Image, torch.Tensor 중 하나여야 함)")
        
        # 필요하다면 전처리
        return image
    
    def detect_pills(self, validated_image):
        """이미지에서 약물(알약) 객체를 탐지합니다.
        Args:
            validated_image: 검증된 이미지

        Returns:
            list: 탐지된 객체 목록
        """
        if self.model_1way is None:
            raise ValueError("1단계 모델이 로드되지 않았습니다.")
        # 이미지가 PIL 이미지인 경우 numpy 배열로 변환
        if isinstance(validated_image, Image.Image):
            validated_image = np.array(validated_image)
        result = self.model_1way(validated_image, verbose=False)

        detected_objects = []
        for box, score, class_id in zip(result[0].boxes.xyxy.cpu().numpy(), result[0].boxes.conf.cpu().numpy(), result[0].boxes.cls.cpu().numpy()):
            detected_objects.append({
                "box": box.tolist(),
                "score": float(score),
                "class_id": int(class_id)
            })
        return detected_objects
    
        # 3단계: 객체 분류
    def classify_pills(self, validated_image, detections):
        CLASS_NAMES = self.database['categorys']
        def search_class_idx(img, box):
            xyxy = box
            # 좌표값을 정수로 변환
            x1, y1, x2, y2 = map(int, xyxy)
            # img가 PIL 이미지면 numpy 배열로 변환
            if isinstance(img, Image.Image):
                img_np = np.array(img)
            else:
                img_np = img
            image = Image.fromarray(img_np[y1:y2, x1:x2])
            input_tensor = self.transform(image).unsqueeze(0).to(self.__device)
            output = self.model_2way(input_tensor)
            probs = F.softmax(output, dim=1)
            _, predicted = torch.max(output, 1)
            cls_idx = predicted.item()
            predicted_class = CLASS_NAMES[cls_idx]
            predicted_prob = probs[0, cls_idx].item()
            return cls_idx, predicted_class, xyxy, probs, predicted_prob
        
        img = validated_image.copy()
        result = []
        for detection in detections:
            box = detection['box']
            cls_idx, predicted_class, xyxy, probs, predicted_prob = search_class_idx(img, box)
            result.append({
                "class_id": cls_idx,
                "class_name": predicted_class,
                "box": xyxy,
                "probabilitie": predicted_prob,
                "class_probabilitie": probs
            })
            # if self.DEBUG_ON:
            #     print(f"Predicted class : [{cls_idx}]  {predicted_class} {xyxy} - Probability: {predicted_prob}")
        return result

    def validate_and_enrich(self, classifications):
        """분류된 객체를 검증하고 DB 조회를 통해 추가 정보를 얻습니다.
        Args:
            classifications: 분류된 객체 목록

        Returns:
            list: 검증 및 추가 정보가 포함된 객체 목록
        """
        category_ids = []
        for classification in classifications:
            category_id = classification['class_name']
            category_ids.append(category_id)

        ddi_result = {}
        if len(category_ids) > 0:
            ddi_result = self.find_ddi(category_ids)
        return ddi_result

    def format_response(self, image, results, classifications):
        def result_to_dict(result):
                # 각 DataFrame을 dict 리스트로 변환
                result_json = {}
                for key in result:
                    # DataFrame이 비어있지 않으면 to_dict로 변환
                    if hasattr(result[key], "to_dict"):
                        result_json[key] = result[key].to_dict(orient="records")
                    else:
                        result_json[key] = result[key]
                return result_json

        for item in classifications:
            if 'class_probabilitie' in item and hasattr(item['class_probabilitie'], 'tolist'):
                item['class_probabilitie'] = item['class_probabilitie'].tolist()
        
        result_dict = result_to_dict(results)
        if type(image) is str:
            result_dict['image_path'] = image

        category = {}
        for classification in classifications:
            category[classification['class_id']] = classification['class_name']

        df_drug = self.database['df_drug'].copy()
        for cls in classifications:
            # print(f"{cls['class_id']:2d} {str(cls['class_name']):20} {cls['box']} {cls['probabilitie']:.4f}")
            # print('category=', category[cls['class_id']])
            df_drug_row = df_drug[df_drug['category_id'].isin([category[cls['class_id']]])]
            df_drug_row.head(10)
            cls['drug_info'] = df_drug_row.to_dict(orient="records")

        result_dict['classifications'] = classifications
        result_json = json.dumps(result_dict, ensure_ascii=False, indent=2)
        
        return result_json

    def init_database(self):
        df_drug = pd.read_pickle(os.path.join(self.data_path, "df_drug.pkl"))
        df_interaction = pd.read_pickle(os.path.join(self.data_path, "df_병용금기약물_20240813.pkl"))
        
        df_drug_sorted = df_drug.sort_values('category_id')
        categorys = df_drug_sorted['category_id'].unique().tolist()
        
        database = {
            "categorys": categorys,
            "df_drug": df_drug,
            "td_interaction": df_interaction
        }
        return database
    
    def load_1way_model(self):
        model_path = os.path.join(self.modeling_path, 
                                            "yolo",
                                            "yolov8m_yolo_noresize_one_class_20250915_0858",
                                            "weights",
                                            "best.pt"
                                            )

        if self.DEBUG_ON:
            print(os.path.exists(model_path), model_path)
        
        model_1way = YOLO(model_path)
        model_1way.to(self.__device)
        model_1way.eval()
        return model_1way
    
    def load_2way_model(self):
        model_path = os.path.join(self.modeling_path, 
                                            "efficientnet_b3",
                                            "efficientnet_b3_experiment_20250915_224641",
                                            "best.pth"
                                            )
        if self.DEBUG_ON:
            print(os.path.exists(model_path), model_path)

        NUM_CLASSES = len(self.database['categorys'])
        model_2way_state, model_2way_info = load_model_dict(model_path)
        model_2way = timm.create_model("efficientnet_b3", pretrained=True, num_classes=NUM_CLASSES)
        model_2way.load_state_dict(model_2way_state)
        model_2way.to(self.__device)
        model_2way.eval()
        return model_2way
    

In [226]:
pillEngine = PillAnalysisEngine()

D:\GoogleDrive\codeit_ai_health_eat
D:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling
D:\GoogleDrive\codeit_ai_health_eat\src\python_modules\data
True D:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling\yolo\yolov8m_yolo_noresize_one_class_20250915_0858\weights\best.pt
True D:\GoogleDrive\codeit_ai_health_eat\src\python_modules\modeling\efficientnet_b3\efficientnet_b3_experiment_20250915_224641\best.pth


In [229]:
impage_path = r'D:\dataset\kaggle_code_it_data\ai04-level1-project.zip.unzip\test_images\1.png'
results_json = pillEngine.analyze_image(impage_path)
results_dict = json.loads(results_json)
#print(results)
print(results_dict['classifications'])
for cls in results_dict['classifications']:
    print(cls['drug_info'])
    #print(json.dumps(cls, ensure_ascii=False, indent=2))
    #print(f"{cls['class_id']:2d} {str(cls['class_name']):20} {cls['box']} {cls['probabilitie']:.4f}")

[{'class_id': 23, 'class_name': 16550, 'box': [562.3856201171875, 62.307830810546875, 951.3316650390625, 478.8019104003906], 'probabilitie': 1.0, 'class_probabilitie': [[1.8785979716273715e-11, 5.0981717084714814e-14, 1.605977347906174e-14, 3.959228431216211e-11, 2.5173649276233334e-10, 1.5635770808522867e-14, 7.220626006831932e-12, 1.9949722429579708e-11, 5.003298990391425e-12, 2.7437948801889434e-08, 1.1718465773620382e-15, 8.50872323987506e-12, 7.643069120308321e-13, 4.710588405345932e-14, 4.473693551027003e-12, 8.657650808828292e-15, 5.3395676281094845e-14, 5.436179634326071e-16, 2.046660511220385e-15, 2.620163577124802e-17, 5.0286202026340074e-17, 9.270383939663507e-13, 8.56810108973427e-12, 1.0, 2.0321987365470862e-14, 6.955080261716873e-11, 3.382351636185138e-14, 1.6107369344570133e-12, 3.4145784869710856e-15, 6.584626100663085e-14, 1.0198142243872388e-12, 4.2016561128781216e-14, 1.3284069211392385e-13, 1.5696085924055136e-12, 8.5955914152884e-14, 2.8784324923591953e-10, 3.75597